In [1]:
import torch

In [2]:
x = torch.normal(0, 1, [31, 455])
h = torch.normal(0, 1, [100, 20, 31])

g = torch.matmul(h, x)
g = torch.heaviside(g, torch.tensor(0.))

w = torch.normal(0, 1, [100, 20, 31]) 

In [3]:
target = torch.ones([455, 1])

In [4]:
g.shape, w.shape

(torch.Size([100, 20, 455]), torch.Size([100, 20, 31]))

# For Loop

In [5]:
res = []
for i in range(g.shape[2]):
    g_i = g[:, :, i]
    r = torch.matmul(g_i, w)
    res.append(r.sum(dim=1))

ew = torch.stack(res)

In [6]:
ew.shape

torch.Size([455, 100, 31])

In [7]:
x.shape

torch.Size([31, 455])

In [8]:
res = []
for i in range(ew.shape[0]):
    x_i = x[:, i]
    ew_i = ew[i, :, :]
    
    res.append(torch.matmul(ew_i, x_i))

r_output = torch.stack(res)

In [9]:
r_output.shape

torch.Size([455, 100])

In [10]:
d = r_output - target

In [11]:
x.shape

torch.Size([31, 455])

In [13]:
g[:, :, None].shape

torch.Size([100, 20, 1, 455])

In [20]:
# res = []
# for i in range(d.shape[1]):
#     x_i = x[:, i]
#     d_i = d[i, :]
#     grad = d_i[:, None] * x_i[None]
#     r = g[:, :, None] * grad[:, None]
    
#     res.append(r)
    
# res = torch.stack(res) #.mean(dim=0)

# Vectorization

In [15]:
ew_vec = torch.tensordot(g, w, dims=([1], [1])).sum(dim=0)
ew_vec.shape

torch.Size([455, 100, 31])

In [16]:
torch.allclose(ew_vec, ew, rtol=1e-03, atol=1e-03)

True

In [17]:
r_output_vec = torch.einsum('ijk,ki->ij', ew_vec, x)

In [18]:
torch.allclose(r_output_vec, r_output, rtol=1e-03, atol=1e-03)

True

In [19]:
d_vec = r_output_vec - target